In [ ]:
%matplotlib inline
# Do not display warnings to prettify the notebook...
import warnings
warnings.simplefilter("ignore")

# Connectivity Analysis Pipeline

This notebook showcases a connectivity analysis pipeline using **EBRAINS** atlas services through **siibra**, and **nilearn**.

## Getting Started

The following steps will help you get started quickly with `siibra` and `nilearn`. We start by trying to import the packages. If not found, they will be installed.

In [ ]:
# import nilearn newest version (make sure it is 0.9.0 or more)
try:
    import nilearn
except:
    !pip install nilearn
    import nilearn
nilearn.__version__

We rely on the `siibra` library to work with EBRAINS human brain atlas and access the *Julich-Brain Probabilistic Cytoarchitectonic Maps*.

In [ ]:
try:
    import siibra
except:
    !pip install siibra
    import siibra
print(siibra.__version__)
assert siibra.__version__ >= "0.4a76"

Alternatively, you can install the most recent development version of siibra directly from github:

```shell
git clone https://github.com/FZJ-INM1-BDA/siibra-python.git
cd siibra-python/
pip install -e .
```

## Pipeline Steps

The pipeline will ideally contains the following steps:

 1. [Load fmri data](#Step-1:-Load-fmri-data)
 2. [Load a parcellation map using siibra](#Step-2:-Load-a-parcellation-map-using-siibra)
 3. [Use Nilearn to extract signals from parcellation and functional data](#Step-3:-Use-Nilearn-to-extract-signals-from-parcellation-and-functional-data)
 4. [Use Nilearn to compute a connectivity matrix](#Step-4:-Use-Nilearn-to-compute-a-connectivity-matrix)
 5. [Fetch streamline counts using siibra](#Step-5:-Fetch-streamline-counts-using-siibra)
 6. [Visual comparison of the matrices](#Step-6:-Visual-comparison-of-the-matrices)

## Step 1: Load fmri data

Fetch one IBC subject in 3mm resolution

In [ ]:
from download_data import download_data
import glob

download_data()
data = {
    'func': glob.glob("**/*.nii.gz", recursive=True),
    'confounds': glob.glob("**/*.txt", recursive=True)
}

## Step 2: Load a parcellation map using siibra

siibra distinguishes a parcellation from its parcelation maps, since the same parcellation can be mapped in different reference spaces (e.g. MNI 152 or BigBrain), and different forms (e.g. as labelled or statistical map). We first choose a parcellation.

In [ ]:
julich_brain = siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_ATLAS_V2_9
print(julich_brain.description)

We then choose the map for this parcellation in MNI152 space. The map is an object which provides access to the corresponding image or surface data, and keeps track of the relationship between map indices and the corresponding regions. It follows a lazy data loading scheme - images or meshes are only retrieved when calling `fetch()`. Maps might be split across multiple volumes: Statistical maps typically use one volume per region, and some labelled maps also split the volume into fragments. For example, the Julich-Brain maximum probability maps are provided in two volume fragments for the left and right hemisphere. When fetching, siibra returns Nifti1Image objects for voxel volumes. 

For this connectivity analysis however, we require a single labelled volume. Therefore we request a compressed parcellation map, where the fragments are merged into a single volume. Note that the compression results in modified label indices, which are automatically managed by the resulting compressed parcellation map object. Compression may inevitably lead to voxel labeling conflicts, which siibra reports with message.

In [ ]:
# the original maximum probability map object with split hemispheres
julich_mpm = julich_brain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.LABELLED
)
print(
    f"The Julich-Brain maximum probability map comes in {len(julich_mpm.fragments)} "
    f"volume fragments: {', '.join(julich_mpm.fragments)}"
)

# single-volume compressed version
julich_mpm_compressed = julich_mpm.compress()
parcellation_map_niimg = julich_mpm_compressed.fetch()

from nilearn import plotting
plotting.view_img(
    parcellation_map_niimg,
    cmap=julich_mpm_compressed.get_colormap(),
    symmetric_cmap=False, colorbar=False
)

**Note:** siibra parcellation maps allow to translate indices into regions and vice versa. Map indices generally define volume index, fragment name, and label index. Typically, index handling is performed automatically in the background.

In [ ]:
# how is v1 left indexed in this map?
index = julich_mpm_compressed.get_index('v1 left')
print(index)

# What is the region mapped with this index?
region1 = julich_mpm_compressed.get_region(index=index)
print(region1)

## Step 3: Use Nilearn to extract signals from parcellation and functional data

In this section we use the nilearn `NiftiLabelsMasker` to extract the signals from the functional dataset and parcellation.

In [ ]:
from IPython.display import Image
Image(filename='masker.png') 

*copyright - Image taken from the nilearn documentation.*

More information on maskers can be found in the <a href="https://nilearn.github.io/stable/manipulating_images/masker_objects.html">nilearn online documentation</a>.

The parcellation and data images have different resolutions, which will have to be handled when computing the signals:

In [ ]:
parcellation_map_niimg.affine

In [ ]:
import nibabel as nib
nib.load(data['func'][0]).affine

In [ ]:
from nilearn.maskers import NiftiLabelsMasker
import numpy as np

# Use NiftiLabelsMasker to extract signals from regions
# Note that the masker will resample the parcellation to
# the data image resolution.
masker = NiftiLabelsMasker(
    labels_img = julich_mpm_compressed.fetch(),
    labels = julich_mpm_compressed.labels,
    background_label=0,  # Default value, for clarity
    resampling_target="data",  # Default value
    standardize=True  # Standardize the signals
) 

# build the time series array
time_series = np.array([
    masker.fit_transform(func, confounds=confounds)
    for func, confounds in zip(data['func'], data['confounds'])
])

Small regions could get wiped out of the label image due to resampling which may results in signals extracted from fewer regions than defined in the input parcellation map.

In [ ]:
# compare the dimensions
print(
    f"We have {time_series.shape[2]} standardized time series "
    f"of length {time_series.shape[1]} for 1 subject, for "
    f"{len(julich_mpm_compressed.regions)} regions in the parcellation map."
)

# Print the left out areas
diff = julich_mpm_compressed.labels - set(masker.labels_)
for i, label in enumerate(diff):
    if i == 0:
        print("\nLeft out regions: ")
    region = julich_mpm_compressed.get_region(volume=0, label=label)
    print(f" - {region.name} (index={label})")

In [ ]:
import matplotlib.pyplot as plt

subject_id = 0
fig = plt.figure(figsize=(12,4))
for i in [0,1,2]:
    # Note: the index in the time series array is a zero-based continuous range,
    # which we need to translate back into the parcellation's label index
    # to identify the corresponding region.
    region = julich_mpm_compressed.get_region(
        volume=0, label=int(masker.labels_[i])
    )
    plt.plot(time_series[subject_id, :, i], label=f"{region.name:30.30}")
plt.legend()
plt.xlim((0, 168))
plt.xlabel("Time", fontsize=15)
plt.title(f"Signals for subject {subject_id} for three regions", fontsize=15)
plt.grid(True)
plt.tight_layout()

## Step 4: Use Nilearn to compute a connectivity matrix

Here we compute the correlation between these time series:

In [ ]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform(time_series)
assert correlation_matrix.shape == (len(data['func']), len(masker.labels_), len(masker.labels_))
print(correlation_matrix.shape)

In order to visualize this matrix, we take the mean across subjects.
Since one subject is used here, this step will only affect the shape of the matrix:

In [ ]:
mean_correlation_matrix = correlation_measure.mean_
assert mean_correlation_matrix.shape == (len(masker.labels_), len(masker.labels_))
print(mean_correlation_matrix.shape)

## Step 5: Fetch streamline counts using siibra

In [ ]:
sc = siibra.features.get(
    julich_brain,
    siibra.features.connectivity.StreamlineCounts
)[0]
sc_mean_matrix = sc.get_matrix()

## Step 6: Visual comparison of the matrices

We can use **Nilearn** to visualize the connectivity, either as a matrix or as a graph

### 6.1 As a matrix

We can plot the matrix with the region names.

In [ ]:
from nilearn.plotting import plot_matrix
# Mask the main diagonal for visualization:
np.fill_diagonal(mean_correlation_matrix, 0)
# matrices are ordered for block-like representation
# For readability show only first 40 labels
start = 0
end = 40
regions = [
    julich_mpm_compressed.get_region(volume=0, label=int(l))
    for l in masker.labels_[start:end]
]
plot_matrix(mean_correlation_matrix[start:end, start:end],
            figure=(16, 16), 
            labels=[r.name for r in regions],
            reorder=False)

In [ ]:
# determine regions in the structural connectivity matrix 
# corresponding to the 40 selected regions in the functional connectivity
sc_regions = [
    r for r in sc_mean_matrix.index
    if any(_.matches(r) for _ in regions)
]
assert len(sc_regions) == 40

# extract the log-scale 40x40 submatrix from the structural connectivity
sc_submatrix = np.log(
    sc_mean_matrix.loc[sc_regions][sc_regions].to_numpy()
)

# plot it
np.fill_diagonal(sc_submatrix, 0)
plot_matrix(sc_submatrix,
            figure=(16, 16), 
            labels=[r.name for r in regions],
            reorder=False)

### 6.2 As a connectome plot

In [ ]:
from nilearn.plotting import plot_connectome, find_parcellation_cut_coords

In [ ]:
# plot the structural connectivity
centroids = sc.compute_centroids(space='mni152')
plot_connectome(sc_mean_matrix, centroids, edge_threshold="99%")

In [ ]:
# grab center coordinates for atlas labels
# !!!!! UGLY HACK ALERT !!!!!
# Pass the resampled label image from the masker to avoid shape errors
coordinates = find_parcellation_cut_coords(labels_img=masker._resampled_labels_img_)
# plot connectome with 99% edge strength in the connectivity
plot_connectome(mean_correlation_matrix,
                coordinates,
                edge_threshold="99%")